## Data Scraping
We will be scraping acne images from https://dermnetnz.org/

In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import shutil

def download_image(image_url, image_name):
    url = image_url
    response = requests.get(url, stream=True)
    with open(image_name, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [40]:
def scrape_dermnet(url):
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    textBlock = soup.find('section', class_="textBlock")
    imgs = textBlock.findAll('img')
    img_dataset = pd.DataFrame([[img.attrs['alt'], "https://dermnetnz.org/" + img.attrs['data-src']] for img in imgs], columns=['alt', 'image_url'])
    img_dataset['image_name'] = [f'data/image_{e}.jpg' for e in range(img_dataset.shape[0])]
    return img_dataset

In [6]:
# Download images
img_dataset = scrape_dermnet("https://dermnetnz.org/topics/acne-face-images")
for row in img_dataset.itertuples():
    download_image(row.image_url, row.image_name)

## Sliding Window

To construct smaller images based on size of sliding window

In [48]:
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [38]:
images = [join('scraped_data', f) for f in listdir('scraped_data') if isfile(join('scraped_data', f))]

In [39]:
# Set size of sliding window
window_size = 50

In [42]:
# Slicing images
for image_path in images:
    img = np.asarray(Image.open(image_path))
    for i in range(0,img.shape[0],window_size):
        for j in range(0,img.shape[1],window_size):
            sub_img = img[i:i+window_size, j:j+window_size, :]
            # Save image
            img_name = image_path.split('/')[-1].split('.')[0]
            Image.fromarray(sub_img).save(f'dataset/{img_name}_{i}_{j}.jpg')


In [53]:
# Create csv for tagging
cropped_images = sorted([f for f in listdir('dataset') if isfile(join('dataset', f))])

In [56]:
tagging_df = pd.DataFrame(cropped_images, columns=['Image File'])

In [57]:
tagging_df.head()

,Image File
0,image_0_0_0.jpg
1,image_0_0_100.jpg
2,image_0_0_150.jpg
3,image_0_0_200.jpg
4,image_0_0_250.jpg


In [58]:
tagging_df.to_csv('labels.csv', index=False)